In [26]:
import ollama
import gradio as gr
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
MODEL = "llama3.2"

## Short Story Creator

In [4]:
# Constants
#user prompt args
GENRES = ["Fiction", "Nonfiction", "Drama", "Poetry", "Fantasy", "Horror", "Mystery", "Science Fiction", "Suspense", "Women's fiction", "Supernatural/Paranormal", "Suspense", "Young adult"]
THEMES = ["Love", "Redemption", "Forgiveness", "Coming of age", "Revenge", "Good vs evil", "Bravery and hardship", "The power of social status","The destructive nature of love", "The fallibility of the human condition"]
WRITING_STYLES = ["Expository", "Narrative", "Descriptive", "Persuasive", "creative"]
TONES = ["Formal", "Formal", "Optimistic", "Worried", "Friendly", "Curious", "Assertive", "Encouraging"]

In [5]:
def get_story_user_prompt(genre, theme, style, tone):
    user_prompt = f"You are looking at genre: {genre}\n"
    user_prompt += f"with the theme: {theme}.\n"
    user_prompt += f"Author's writingStyle is: {style}.\n"
    user_prompt += f"Tone of the story is: {tone}."
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [8]:
def create_short_story(genre, theme, style, tone):
    response = ollama.chat(model=MODEL, messages=[
            {"role": "system", "content": story_system_prompt},
            {"role": "user", "content": get_story_user_prompt(genre, theme, style, tone)}
          ])
    return response['message']['content']

In [6]:
story_system_prompt = "You are an creative story writing assistant that takes input like genre, theme, or character type \
and generates a short story."

In [9]:
response = create_short_story("Fiction", "Bravery and hardship", "Creativity", "Encouraging")
print(response)

**The Ember of Resilience**

In the scorching desert, where dunes stretched like golden seas and sunbeams scorched the earth, a young nomad named Aisha traversed the unforgiving terrain. Her eyes, like two dark stars, shone bright with determination, for she was on a quest to fulfill an unyielding dream.

Aisha's village had been ravaged by a brutal sandstorm that left her family homeless and her heart shattered. The loss of her mother, who had taught her the ancient ways of their people, still lingered like an open wound. Yet, Aisha refused to let despair consume her. Instead, she channeled her grief into the flame of resilience that burned within.

As she walked, the blistering sun beat down upon her, relentless in its ferocity. The wind howled, threatening to snuff out the ember of hope that flickered within her chest. Yet Aisha persevered, fueled by the memories of her mother's words: "A brave heart is not one that shuns hardship, but one that finds strength in the face of adversit

In [10]:
# Gradio UI
gr.Interface(fn=create_short_story, inputs=[gr.Dropdown(GENRES, label="genre", info="select your story genre"),
                                gr.Dropdown(THEMES, label="theme", info="select your story theme"),
                                gr.Dropdown(WRITING_STYLES, label="style", info="select writingStyle"),
                                gr.Dropdown(TONES, label="tone", info="select the tone")
             ], outputs="textarea").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Building Chatbot

In [18]:
# Constants
system_message = "You are an optimistic assistant and will respond to user queries with positivity"


In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = ollama.chat(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
         if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response

In [25]:
# gradio UI
gr.ChatInterface(fn=chat,
                 type='messages',
                 chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type Question Here", container=False, scale=3),
    title="helpful Assistant",
    description="Ask me a question and I will give you an helpful answer",
    theme="Glass",
    examples=["Will the sky fall today?", "were dinosaur existed before?", "Advantages of work life balance", "Is yes yes and a no a no"]).launch()

/Users/sivaram/Developer/SDS/SDS-CP019-local-ai-writing-assistant/.venv/lib/python3.12/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/Users/sivaram/Developer/SDS/SDS-CP019-local-ai-writing-assistant/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:222: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


## Website Summarizer

In [27]:
 # A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [28]:
# Let's try one out

espn = Website("https://espncricinfo.com")
print(espn.title)
print(espn.text)

Today's Cricket Match | Cricket Update | Cricket News | ESPNcricinfo
Matches
(
26
)
BBL 2024
(
3
)
BPL 2024
(
2
)
Super Smash
(
2
)
Women's Super Smash
(
1
)
Vijay Hazare Trophy
(
18
)
Previous
RESULT
•
17th Match
•
BBL 2024
•
T20
•
Adelaide
AS
142/8
PS
(14.3/20 ov, T:143)
146/3
Scorchers won by 7 wickets (with 33 balls remaining)
Schedule
Table
Report
Series
RESULT
•
4th Match
•
BPL 2024
•
T20
•
Mirpur
RAR
155/6
SYS
(20 ov, T:156)
121/9
Rangpur won by 34 runs
Schedule
Table
Series
RESULT
•
3rd Match
•
BPL 2024
•
T20
•
Mirpur
KT
203/4
Kings
(18.5/20 ov, T:204)
166
Khulna won by 37 runs
Schedule
Table
Series
RESULT
•
4th Match
•
Super Smash
•
T20
•
Alexandra
OTAGO
154/8
CD
(18.5/20 ov, T:155)
158/3
Central D won by 7 wickets (with 7 balls remaining)
Schedule
Table
Series
RESULT
•
4th Match
•
Women's Super Smash
•
Alexandra
CD-W
89/9
OTG-W
(19.5/20 ov, T:90)
92/9
Otago Women won by 1 wicket (with 1 ball remaining)
Schedule
Table
Series
RESULT
•
Group D
•
Vijay Hazare Trophy
•
List A
•
Vi

In [29]:
# Define our system prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [30]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [31]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [32]:
def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [33]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://cnn.com")

Here is a summary of the topics and articles available on CNN.com:

**Top Stories**

* Trump's energy strategy faces challenges, including opposition from environmental groups
* The Biden administration's plan to ban natural gas exports is being pushed back by the former president
* Climate change is a growing concern, with scientists warning of devastating consequences if action is not taken
* The Ukraine-Russia War continues to escalate, with tensions between the two nations rising

**Business**

* The US economy is experiencing a slowdown, with GDP growth slowing down in 2024
* The Federal Reserve is considering raising interest rates to combat inflation
* Tech companies are facing increased scrutiny over their environmental impact and social responsibility

**Health**

* Alzheimer's disease is a growing concern, with new research suggesting that music may be a effective treatment
* Human composting is becoming increasingly popular as an earth-friendly alternative to traditional burial methods
* Mental health experts are warning of the dangers of social media addiction and the importance of self-care

**Entertainment**

* Taylor Swift kisses Travis Kelce after win in the NFL playoffs
* Beluga whales are being studied for their unique behavior and intelligence
* The 1980s soda Jolt Cola is making a comeback with an extra kick of caffeine

**Sports**

* The College Football Playoff is underway, with top teams competing for a spot in the national championship game
* The NBA playoffs are heating up, with some of the league's biggest stars on display
* Soccer fans around the world are gearing up for the World Cup, which is set to kick off soon

**Science**

* Scientists have discovered a new species of ancient fungus that may hold the key to creating vegan cheese
* Researchers are exploring the potential of microplastics in the ocean and how they can be removed
* The environment is being affected by climate change, with rising temperatures and extreme weather events becoming more frequent

**Politics**

* The SCOTUS is set to hear a landmark case on abortion rights, which could have significant implications for the country
* Congress is grappling with the challenges of balancing individual freedom with collective security in the face of terrorism threats
* The Biden administration's plan to address income inequality is being debated by lawmakers and economists

**Innovate**

* Google has unveiled a new AI tool that uses image prompts instead of text, which could revolutionize the way we interact with technology
* A new type of sponge made from squid bones may hold the key to removing microplastics from our oceans
* The future of work is being shaped by advances in technology and automation, which are changing the nature of employment and education.

In [35]:
display_summary("https://nfl.com")


**Summary**
================

### Overview
The official website of the National Football League (NFL) provides comprehensive coverage of all things football.

### Features

* **News**: Up-to-date news and announcements about NFL teams, players, and events.
* **Games**: Live scores, schedules, and game highlights for current and past games.
* **Fantasy**: Resources and tools for fantasy football leagues.
* **Shop**: Official merchandise store for NFL teams and gear.

### Technical Notes
--------------------

* The website uses a responsive design that adapts to different screen sizes and devices.
* It includes various accessibility features such as closed captions, audio descriptions, and high contrast mode.
* The site recommends using the latest versions of supported browsers (Chrome, Edge 80+, Firefox, Safari) for an optimal viewing experience.

### Important Announcement
-------------------------

* **Browser Compatibility**: A note is displayed warning users about browser compatibility issues and recommending the use of supported browsers.